In [9]:
import psycopg2
import urllib
import json
import velovUtils
import pandas as pd
from datetime import timedelta
import datetime
import seaborn as sns
from scipy.stats import exponweib
url = 'https://download.data.grandlyon.com/ws/rdata/jcd_jcdecaux.jcdvelov/all.json'
conn = psycopg2.connect(database = "velov", 
                        user = "dev", 
                        password = "velov", 
                        host = "localhost" ,
                        port = "5432")

In [33]:
# Create sql statement for retrieve data from database for given station and given hour
station = 11
hour = 9
sqlSelect = "select distinct last_update, available_bike_stands from velovdata where station_name =" + str(
        station) + " and extract(HOUR FROM last_update) =" + str(hour) + " order by last_update"

In [34]:
# Retrieve all data for station at given hour into a pandas.dataFrame
cur = conn.cursor()
cur.execute(sqlSelect)
data = cur.fetchall()
dataAsDF = pd.DataFrame(data)
labels = ['last_update', 'avail_bike_stands']
for columns_label in range(0, len(labels)):
    dataAsDF = dataAsDF.rename(columns={columns_label: labels[columns_label]})

In [35]:
dataAsDF

""


In [25]:
# First add to dataAsDF column for duration between two update
duration = [timedelta(seconds=0)] + list(dataAsDF.last_update[i + 1] - dataAsDF.last_update[i] 
                                         for i in range(0, len(dataAsDF) - 1))
dataAsDF = pd.concat([dataAsDF, pd.DataFrame(duration)], axis=1)
dataAsDF = dataAsDF.rename(columns={0: 'duration_since_last_update'})

In [26]:
dataAsDF

,last_update,avail_bike_stands,duration_since_last_update
0,2022-05-19 06:57:43,10,0 days 00:00:00
1,2022-05-19 06:59:13,30,0 days 00:01:30
2,2022-05-19 07:05:11,30,0 days 00:05:58
3,2022-05-19 07:15:16,30,0 days 00:10:05
4,2022-05-19 07:26:41,29,0 days 00:11:25
5,2022-05-19 07:27:47,28,0 days 00:01:06
6,2022-05-19 07:29:14,29,0 days 00:01:27
7,2022-05-19 07:40:18,30,0 days 00:11:04
8,2022-05-19 07:42:19,29,0 days 00:02:01
9,2022-05-19 07:45:53,28,0 days 00:03:34


In [18]:
dataAsDF['real_update'] = dataAsDF['duration_since_last_update'].apply(lambda update: 
                                                                       update <= timedelta(minutes=10))

In [ ]:
dataAsDF.to_csv('test.csv')

In [ ]:
sns.displot(data=dataAsDF.loc[dataAsDF['real_update']==True]['avail_bike_stands'],
            kde=True)

In [ ]:
weibulltheo = exponweib.rvs(1, 0.5, loc=0, scale=1, size=200)
sns.displot(data=weibulltheo,
           kde=True)